# Analyse de sentiments avec Word2Vec

# Les données

In [ ]:
import numpy as np


In [ ]:
###########################################
### Just run this cell to load the data ###
###########################################

import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence

def load_data(percentage_of_sentences=None):
    train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], batch_size=-1, as_supervised=True)

    train_sentences, y_train = tfds.as_numpy(train_data)
    test_sentences, y_test = tfds.as_numpy(test_data)

    # Take only a given percentage of the entire data
    if percentage_of_sentences is not None:
        assert(percentage_of_sentences> 0 and percentage_of_sentences<=100)

        len_train = int(percentage_of_sentences/100*len(train_sentences))
        train_sentences, y_train = train_sentences[:len_train], y_train[:len_train]

        len_test = int(percentage_of_sentences/100*len(test_sentences))
        test_sentences, y_test = test_sentences[:len_test], y_test[:len_test]

    X_train = [text_to_word_sequence(_.decode("utf-8")) for _ in train_sentences]
    X_test = [text_to_word_sequence(_.decode("utf-8")) for _ in test_sentences]

    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = load_data(percentage_of_sentences=10)


Dans l'exercice précédent, vous avez entraîné une représentation Word2vec et converti toutes vos phrases d'entraînement afin de les introduire dans un RNN, comme le montre la première étape de cette figure :

<img src="word2vec_representation.png" width="400px" />



❓ Refaites exactement ce que vous avez fait dans l'exercice précédent. D'abord, entraînez un modèle word2vec (avec les arguments que vous voulez) sur votre phrase d'entraînement. Enregistrez-le dans la variable `word2vec`.

Réutilisons les fonctions de l'exercice précédent pour convertir vos données d'entraînement et de test en quelque chose que vous pouvez introduire dans un RNN.

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence(word2vec, sentence):
    pass

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    pass

# Embed the training and test sentences



# Pad the training and test embedded sentences


☝️ Pour être sûr que cela a fonctionné, vérifions ce qui suit pour `X_train_pad` et `X_test_pad` :
- ce sont des tableaux numpy
- ils sont tridimensionnels
- la dernière dimension est la taille de votre espace d'embedding word2vec (vous pouvez l'obtenir avec `word2vec.wv.vector_size`)
- la première dimension est la taille de vos `X_train` et `X_test`.

✅ **Bonne pratique** ✅ De tels tests sont très importants ! Non seulement dans cet exercice, mais dans les applications de la vie réelle. Ils évitent de trouver des erreurs trop tard et de les laisser se propager dans tout le notebook.

In [ ]:
# TEST ME
for X in [X_train_pad, X_test_pad]:
    assert type(X) == np.ndarray
    assert X.shape[-1] == word2vec.wv.vector_size


assert X_train_pad.shape[0] == len(X_train)
assert X_test_pad.shape[0] == len(X_test)


# Baseline

Il est toujours bon d'avoir un modèle très simple pour tester votre propre modèle - pour être sûr que vous faites quelque chose de mieux qu'un algorithme très simple.

❓ Quelle est votre accuracy de base ? Dans ce cas, votre base peut être de prédire le label qui est le plus présent dans `y_train` (bien sûr, si le jeu de données est équilibré, la précision de base est 1/n où n est le nombre de classes - 2 ici).

# Le modèle

❓ Créez un RNN avec les couches suivantes :
- une couche `Masking`
- une `LSTM` avec 20 unités et la fonction d'activation `tanh`.
- une couche `Dense` avec 10 unités
- une couche de sortie qui dépend de votre tâche (`sigmoid` parce qu'il s'agit d'un problème de classification)

Ensuite, compilez votre modèle (utiliser `rmsprop` comme optimiseur - au moins pour commencer).

In [ ]:
def init_model():
    pass


❓ Entrainer le modèle sur vos données incorporées et paddées - n'oubliez pas le critère d'arrêt précoce.

❗ **Remarque** ❗ Votre précision dépendra grandement de votre corpus d'entraînement. Ici, assurez-vous simplement que votre performance est supérieure au modèle de base (ce qui devrait être le cas même si vous n'avez chargé que 20% des données IMDB initiales).

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping


❓ Évaluer votre modèle sur l'ensemble de test

# Word2Vec - Transfer Learning

Votre précision, bien que supérieure à celle du modèle de base, peut être assez faible. Il existe de nombreuses options pour l'améliorer, comme le nettoyage des données et l'amélioration de la qualité de l'intégration.

Nous ne nous pencherons pas ici sur les stratégies de nettoyage des données. Essayons d'améliorer la qualité de notre embedding. Mais au lieu de simplement charger un corpus plus important, pourquoi ne pas profiter de l'embedding que d'autres ont entraîné ? En effet, la qualité d'un embedding, c'est-à-dire la proximité des mots, peut être dérivée de différentes tâches. C'est exactement ce qu'est l'apprentissage par transfert.



❓ Lister les différents modèles disponibles dans word2vec grâce à cela :

In [ ]:
import gensim.downloader as api
print(list(api.info()['models'].keys()))


❓ **Question** ❓ Charger un des embeddings de word2vec pré-entraînés. 

Vous pouvez le faire avec `api.load(the-model-of-your-choice)`, et le stocker dans `word2vec_transfer`

<details>
    <summary>💡 Piste</summary>
    
Le modèle `glove-wiki-gigaword-50` est un bon candidat pour commencer car il est plus petit (65 MB).

</details>

❓ Vérifier la taille du vocabulaire, mais aussi la taille de l'espace d'embedding

❓ Faire l'embedding de `X_train` et `X_test`, comme dans la première question il y a les fonctions pour le faire ! (Il y a une légère différence dans la fonction `embed_sentence_with_TF`)

In [ ]:
# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence_with_TF(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec:
            embedded_sentence.append(word2vec[word])

    return np.array(embedded_sentence)

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []

    for sentence in sentences:
        embedded_sentence = embed_sentence_with_TF(word2vec, sentence)
        embed.append(embedded_sentence)

    return embed

# Embed the training and test sentences
X_train_embed_2 = embedding(word2vec_transfer, X_train)
X_test_embed_2 = embedding(word2vec_transfer, X_test)


❓ Faire le padding et enregistrer dans `X_train_pad_2` and `X_test_pad_2`.

❓ Réinitialisez un modèle et entraînez-le sur vos nouvelles données !  Évaluez-le sur votre ensemble de test et comparez-le à votre précision précédente.

❗ **Remarque** ❗ L'entraînement peut prendre un certain temps. Vous pouvez simplement calculer 10 epochs (ce n'est **pas** une bonne pratique, il s'agit juste de ne pas attendre trop longtemps).

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping


Parce que votre nouveau word2vec a été entraîné sur un grand corpus, il a une représentation pour de nombreux mots ! Bien plus qu'avec votre petit ensemble de données, d'autant plus que vous avez éliminé les mots qui n'étaient pas présents plus d'un certain nombre de fois dans l'ensemble d'entraînement. Pour cette raison, vous avez beaucoup plus de mots intégrés dans vos ensembles de formation et de test, ce qui rend chaque itération plus longue qu'auparavant